In [1]:
import pandas as pd
import numpy as np
import re # пришлось использовать, т.к. при импорте датасета, сохранённого из тетрадки 2го вебинара, некооректно подтягивались значения
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
data = pd.read_csv('../../retail_train.csv')
data.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [6]:
data_train.describe()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
count,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06
mean,1.271764e+03,3.294526e+10,3.491402e+02,2.791955e+06,1.006171e+02,3.095110e+00,2.992061e+03,-5.393603e-01,1.562467e+03,5.056328e+01,-1.646478e-02,-2.915685e-03
std,7.269816e+02,3.964679e+09,1.676271e+02,3.673791e+06,1.153002e+03,4.196106e+00,8.693638e+03,1.236080e+00,4.025741e+02,2.394798e+01,2.179563e-01,3.995998e-02
min,1.000000e+00,2.698485e+10,1.000000e+00,2.567100e+04,0.000000e+00,0.000000e+00,1.000000e+00,-1.300200e+02,0.000000e+00,1.000000e+00,-5.593000e+01,-7.700000e+00
25%,6.540000e+02,3.003546e+10,2.080000e+02,9.167670e+05,1.000000e+00,1.270000e+00,3.300000e+02,-6.900000e-01,1.306000e+03,3.000000e+01,0.000000e+00,0.000000e+00
50%,1.271000e+03,3.214976e+10,3.510000e+02,1.027068e+06,1.000000e+00,2.000000e+00,3.700000e+02,-2.000000e-02,1.615000e+03,5.100000e+01,0.000000e+00,0.000000e+00
75%,1.914000e+03,3.433825e+10,4.940000e+02,1.131351e+06,1.000000e+00,3.490000e+00,4.220000e+02,0.000000e+00,1.846000e+03,7.100000e+01,0.000000e+00,0.000000e+00
max,2.500000e+03,4.129777e+10,6.350000e+02,1.782923e+07,8.963800e+04,8.400000e+02,3.428000e+04,3.990000e+00,2.359000e+03,9.100000e+01,0.000000e+00,0.000000e+00


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [7]:
# создаём сводную таблицу с показателями валовых продаж в деньгах по каждому объекту (товару)
popularity = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popularity.sort_values(by='sales_value', ascending=False, inplace=True)
popularity.rename(columns={'sales_value': 'gross_sales'}, inplace=True)

# подсчитываем сумму продаж ВСЕХ товаров
total_sum = popularity['gross_sales'].sum()

# посчитаем коэф-т по каждому товару
popularity['weight']=popularity['gross_sales']/total_sum
popularity.head(10)

,item_id,gross_sales,weight
55470,6534178,447799.94,0.063498
55430,6533889,40483.34,0.005741
28895,1029743,35764.66,0.005071
55465,6534166,30170.77,0.004278
34707,1082185,26029.96,0.003691
16343,916122,25861.84,0.003667
55421,6533765,24764.68,0.003512
37343,1106523,24293.86,0.003445
25064,995242,22937.19,0.003252
51478,5569230,20494.05,0.002906


In [8]:
# создадим вектор весов
item_weights = popularity['weight']

# проверим, что сумма весов по всем товарам = 1
item_weights.sum()

1.0

In [9]:
# создаём вектор объектов (товаров)
items_id = popularity['item_id']

In [10]:
# функция взвешенных k случайных рекомендаций. На вход: 1) вектор товаров (объектов) 2) вектор весов 3) k=членов
def weighted_random_recommendation(items_id, items_weights, n=5):   
    recs = np.random.choice(items_id, size=n, p=items_weights, replace=False)
    return recs.tolist()

In [11]:
recommendations = weighted_random_recommendation(items_id, item_weights, n=5)
print(f'5 случайных взвешенных рекомендаций:\n',recommendations)

5 случайных взвешенных рекомендаций:
 [914512, 5568789, 864893, 1028296, 1002771]


In [12]:
# проверим, что за веса у выбранных нами на предыдущем шаге item_id
popularity.loc[popularity['item_id'].isin (recommendations)]

,item_id,gross_sales,weight
25924,1002771,1730.07,0.000245
10695,864893,1313.11,0.000186
51399,5568789,352.79,0.000050
28732,1028296,114.54,0.000016
16174,914512,69.95,0.000010


In [13]:
# ресурсы про настройку взвешенной случайной выборки
# https://pynative.com/python-weighted-random-choices-with-probability/
# https://www.python-course.eu/weighted_choice_and_sample.php
# https://www.geeksforgeeks.org/how-to-get-weighted-random-choice-in-python/
# https://stackoverflow.com/questions/3679694/a-weighted-version-of-random-choicea

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [45]:
def precision_at_k(recommended_list, bought_list, k=5):
        
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #print(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list#[:k]
    
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [16]:
# загрузка данных из файла
result = pd.read_pickle('predictions_basic.pickle')
result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[14026177, 973880, 1089820, 949556, 852282]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13073225, 1388846, 845538, 5996651, 222031]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[868389, 833260, 10121970, 868907, 9686413]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[10312288, 13943490, 45800, 858687, 10210707]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1104245, 948832, 984591, 884118, 8119018]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]"


In [17]:
# сформируем список названий столбцов, в которых содержатся рекомендации
col_names = result.columns[2:]
col_names = col_names[:6]
col_names

Index(['random_recommendation', 'popular_recommendation', 'itemitem', 'cosine',
       'tfidf', 'own_purchases'],
      dtype='object')

In [18]:
# создаём новые столбцы и рассчитываем для каждого юзера метрику "precision at k"
for col_name in col_names:
    new_name = col_name + '_prec'
    print(col_name, new_name)
    result[new_name ]=None
    for index, row in result.iterrows():
        acts = row['actual']
        recs = row[col_name]
        prec = precision_at_k(recs, acts,5)
        result[new_name][index] = prec

random_recommendation random_recommendation_prec


/Users/artem/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


popular_recommendation popular_recommendation_prec
itemitem itemitem_prec
cosine cosine_prec
tfidf tfidf_prec
own_purchases own_purchases_prec


In [19]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,random_recommendation_prec,popular_recommendation_prec,itemitem_prec,cosine_prec,tfidf_prec,own_purchases_prec
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[14026177, 973880, 1089820, 949556, 852282]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]",0,0.2,0.4,0.2,0.2,0.4
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13073225, 1388846, 845538, 5996651, 222031]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]",0,0,0,0,0,0
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[868389, 833260, 10121970, 868907, 9686413]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]",0,0,0.2,0,0,0
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[10312288, 13943490, 45800, 858687, 10210707]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]",0,0.2,0.2,0.4,0.4,0.2
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1104245, 948832, 984591, 884118, 8119018]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]",0,0.4,0.2,0.2,0.2,0.4


In [20]:
# выводим среднюю точность по каждой методике
for col_name in col_names:
    prec_col_name = col_name+"_prec"
    print(prec_col_name, result[prec_col_name].mean())
#     print(f'метрика по {col_name}: {result[col_name].sum()}, среднее {result[col_name].mean}')

random_recommendation_prec 0.0013712047012732617
popular_recommendation_prec 0.15523996082272082
itemitem_prec 0.13692458374142857
cosine_prec 0.13290891283055686
tfidf_prec 0.1389813907933383
own_purchases_prec 0.17969311132876015


### лучшим алгоритмом оказался own_purchases и popular_recommendation

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов

#### план действий:
1. Создадим датафрейм с топ-5000 (по продажам в деньгах) товаров
2. Вычислим веса для каждого товара
2. "Прогоним" бейзлайны с учётом топ-5000


In [33]:
# Создадим датафрейм с топ-5000 (по продажам в деньгах) товаров
group_data = data_train.groupby('item_id')['sales_value'].sum().reset_index()
top5000 = group_data.sort_values(by='sales_value', ascending=False) [:5000]
top5000

,item_id,sales_value
55470,6534178,447799.94
55430,6533889,40483.34
28895,1029743,35764.66
55465,6534166,30170.77
34707,1082185,26029.96
...,...,...
72352,12188446,265.29
37958,1112387,265.26
23584,981716,265.17
51978,5575861,265.14


In [34]:
# посчитаем веса внутри этих 5000 товаров таким образом, чтобы сумма продаж всех товаров = 1
gross_sales = top5000['sales_value'].sum()
gross_sales

4495663.440000001

In [35]:
# создаём столбец весов: делим значение продаж на общуюю сумму продаж:
top5000['weight']=top5000['sales_value']/gross_sales

In [36]:
top5000

,item_id,sales_value,weight
55470,6534178,447799.94,0.099607
55430,6533889,40483.34,0.009005
28895,1029743,35764.66,0.007955
55465,6534166,30170.77,0.006711
34707,1082185,26029.96,0.005790
...,...,...,...
72352,12188446,265.29,0.000059
37958,1112387,265.26,0.000059
23584,981716,265.17,0.000059
51978,5575861,265.14,0.000059


### Random Recommender Baseline

In [37]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [66]:
# Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)
result1 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result1.columns=['user_id', 'actual']
result1.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [67]:
%%time

items = top5000.item_id.unique()

result1['random_recommendation'] = result1['user_id'].apply(lambda x: random_recommendation(items, n=5))
result1.head(10)

CPU times: user 445 ms, sys: 42.4 ms, total: 488 ms
Wall time: 548 ms


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[7442353, 1080331, 845208, 921744, 925178]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12386079, 1046587, 6602327, 9487286, 1009333]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[7024847, 1118046, 1128016, 995151, 941198]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[882441, 955687, 827047, 1009128, 996955]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5568249, 839162, 5569845, 1048200, 995311]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[886395, 835530, 861821, 1082955, 1052512]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[945244, 882308, 942077, 1083733, 835558]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[983106, 964521, 5981267, 1023815, 917393]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[939900, 1132956, 9834636, 888543, 961756]"
9,16,"[1062973, 1082185, 13007710]","[8203834, 833940, 1121384, 941830, 902640]"


In [68]:
result1['precision_at_k'] = None
for index, row in result1.iterrows():
        acts = row['actual']
        recs = row['random_recommendation']
        prec = precision_at_k(recs, acts,5)
        result1['precision_at_k'][index] = prec

/Users/artem/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
result1.head(5)

,user_id,actual,random_recommendation,precision_at_k
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[7442353, 1080331, 845208, 921744, 925178]",0
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12386079, 1046587, 6602327, 9487286, 1009333]",0
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[7024847, 1118046, 1128016, 995151, 941198]",0
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[882441, 955687, 827047, 1009128, 996955]",0
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5568249, 839162, 5569845, 1048200, 995311]",0


In [70]:
# посчитаем среднюю точность первым способом
result1['precision_at_k'].mean()

0.0055827619980411336

In [71]:
# посчитаем среднюю точность вторым способом
result1.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()

0.0055827619980411336

### Weighted Random Recommender

In [72]:
# создадим вектор весов
item_weights = top5000['weight']

# проверим, что сумма весов по всем товарам = 1
item_weights.sum()

0.9999999999999998

In [73]:
items = top5000.item_id.unique()

result1['weighted_random'] = result1['user_id'].apply(lambda x: weighted_random_recommendation(items, item_weights,n=5))
result1.head(10)

,user_id,actual,random_recommendation,precision_at_k,weighted_random
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[7442353, 1080331, 845208, 921744, 925178]",0,"[1137775, 6534178, 972416, 982743, 1019637]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12386079, 1046587, 6602327, 9487286, 1009333]",0,"[985999, 1053875, 1065593, 881047, 1015539]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[7024847, 1118046, 1128016, 995151, 941198]",0,"[1077555, 1133018, 916122, 8203599, 983584]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[882441, 955687, 827047, 1009128, 996955]",0,"[1128812, 1130375, 954495, 938700, 983316]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5568249, 839162, 5569845, 1048200, 995311]",0,"[6979437, 825541, 6534178, 1098248, 1093030]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[886395, 835530, 861821, 1082955, 1052512]",0,"[847573, 847270, 970747, 876984, 834826]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[945244, 882308, 942077, 1083733, 835558]",0,"[1133556, 882305, 1003600, 862582, 6534178]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[983106, 964521, 5981267, 1023815, 917393]",0,"[1131374, 845208, 6534166, 12352248, 6533765]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[939900, 1132956, 9834636, 888543, 961756]",0,"[1056509, 1082955, 882489, 12301839, 1048507]"
9,16,"[1062973, 1082185, 13007710]","[8203834, 833940, 1121384, 941830, 902640]",0,"[1065538, 857006, 8090521, 986912, 12263492]"


In [74]:
result1['precision_at_k2'] = None
for index, row in result1.iterrows():
        acts = row['actual']
        recs = row['weighted_random']
        prec = precision_at_k(recs, acts,5)
        result1['precision_at_k2'][index] = prec

/Users/artem/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [75]:
result1.head()

,user_id,actual,random_recommendation,precision_at_k,weighted_random,precision_at_k2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[7442353, 1080331, 845208, 921744, 925178]",0,"[1137775, 6534178, 972416, 982743, 1019637]",0
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12386079, 1046587, 6602327, 9487286, 1009333]",0,"[985999, 1053875, 1065593, 881047, 1015539]",0
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[7024847, 1118046, 1128016, 995151, 941198]",0,"[1077555, 1133018, 916122, 8203599, 983584]",0
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[882441, 955687, 827047, 1009128, 996955]",0,"[1128812, 1130375, 954495, 938700, 983316]",0.2
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5568249, 839162, 5569845, 1048200, 995311]",0,"[6979437, 825541, 6534178, 1098248, 1093030]",0


In [76]:
# посчитаем среднюю точность
result1['precision_at_k2'].mean()

0.03379040156709126

#### Точность выросла в 6 раз!

### Popularity-based recommendation

In [97]:
# найдём 5 самых популярных товаров из топ5000 (да и какая разница, они всё равно будут топ5)
top5_recs = top5000.head(5)
top5_recs = top5_recs['item_id']
top5_recs 

55470    6534178
55430    6533889
28895    1029743
55465    6534166
34707    1082185
Name: item_id, dtype: int64

In [104]:
result1['popular'] = result1['user_id'].apply(lambda x: top5_recs.values)
result1.head(5)

,user_id,actual,random_recommendation,precision_at_k,weighted_random,precision_at_k2,popular
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[7442353, 1080331, 845208, 921744, 925178]",0,"[1137775, 6534178, 972416, 982743, 1019637]",0,"[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12386079, 1046587, 6602327, 9487286, 1009333]",0,"[985999, 1053875, 1065593, 881047, 1015539]",0,"[6534178, 6533889, 1029743, 6534166, 1082185]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[7024847, 1118046, 1128016, 995151, 941198]",0,"[1077555, 1133018, 916122, 8203599, 983584]",0,"[6534178, 6533889, 1029743, 6534166, 1082185]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[882441, 955687, 827047, 1009128, 996955]",0,"[1128812, 1130375, 954495, 938700, 983316]",0.2,"[6534178, 6533889, 1029743, 6534166, 1082185]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5568249, 839162, 5569845, 1048200, 995311]",0,"[6979437, 825541, 6534178, 1098248, 1093030]",0,"[6534178, 6533889, 1029743, 6534166, 1082185]"


In [108]:
result1['precision_at_k3'] = None
for index, row in result1.iterrows():
        acts = row['actual']
        recs = row['popular']
        prec = precision_at_k(recs, acts,5)
        result1['precision_at_k3'][index] = prec

/Users/artem/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [109]:
result1.head()

,user_id,actual,random_recommendation,precision_at_k,weighted_random,precision_at_k2,popular,precision_at_k3
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[7442353, 1080331, 845208, 921744, 925178]",0,"[1137775, 6534178, 972416, 982743, 1019637]",0.2,"[6534178, 6533889, 1029743, 6534166, 1082185]",0.2
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12386079, 1046587, 6602327, 9487286, 1009333]",0,"[985999, 1053875, 1065593, 881047, 1015539]",0,"[6534178, 6533889, 1029743, 6534166, 1082185]",0
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[7024847, 1118046, 1128016, 995151, 941198]",0,"[1077555, 1133018, 916122, 8203599, 983584]",0,"[6534178, 6533889, 1029743, 6534166, 1082185]",0
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[882441, 955687, 827047, 1009128, 996955]",0,"[1128812, 1130375, 954495, 938700, 983316]",0.2,"[6534178, 6533889, 1029743, 6534166, 1082185]",0.2
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5568249, 839162, 5569845, 1048200, 995311]",0,"[6979437, 825541, 6534178, 1098248, 1093030]",0.4,"[6534178, 6533889, 1029743, 6534166, 1082185]",0.4


In [110]:
# посчитаем среднюю точность
result1['precision_at_k3'].mean()

0.15523996082272082

#### Точность существенно увеличилась

### Item-item tuning